In [7]:
%matplotlib qt

from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import cv2
from skimage import data, util
from skimage.measure import label, regionprops
import numpy as np
import math
import os

In [8]:
def muCalculator(pointsArray):
    mu = np.mean(pointsArray, axis=0)
    return mu

def sigmaCovCalculator(pointsArray):
    mu = muCalculator(pointsArray)
    sigma = (1/pointsArray.shape[0]) * np.dot((pointsArray - mu).T , (pointsArray - mu))
    return sigma

def gaussian(pointsArray, mu, sigma):
    gaussian = np.zeros((pointsArray.shape[0],1))
    for pointIndex, point in enumerate(pointsArray):
        difference = (point-mu).T
        multiplier = 1/ ( math.pow((2*math.pi),(3/2)) * (math.pow( np.linalg.det(sigma) , (1/2) ) ) )
        exponent = math.exp(-0.5 * np.dot(np.dot( difference.T, np.linalg.inv(sigma)), difference ))
        output = multiplier * exponent
        gaussian[pointIndex] = output
    return gaussian

def calculateGMMProbabilities(inputImage, clusters):
    GMMOutput = np.zeros((inputImage.shape[0], 1))
    for cluster in clusters:
        simpleGaussianOutput = gaussian(inputImage, cluster["mu"], cluster["sigma"])
        GMMOutput = GMMOutput + ((1/clusters.shape[0]) * simpleGaussianOutput)
    return GMMOutput

In [18]:
folder = "Test_Set"
clusters = np.load('clusters.npy', allow_pickle=True)
# imageFolderpath = "Test_Set/"

for filenameIndex, filename in enumerate(os.listdir(folder)):
    print("Calculating probabilities for file number: " + str(filenameIndex) + "...")
    # Reading images as RGB and HSV
    # img_RGB = cv2.cvtColor( cv2.imread(os.path.join(folder,filename), 1) , cv2.COLOR_BGR2RGB)
    # img_HSV = cv2.cvtColor( cv2.imread(os.path.join(folder,filename), 1) , cv2.COLOR_BGR2HSV)

    imageName = os.fsdecode(filename)
    path = "Test_Set/" + str(imageName)
    img_RGB = cv2.cvtColor( cv2.imread(path, 1) , cv2.COLOR_BGR2RGB)
    img_HSV = cv2.cvtColor( cv2.imread(path, 1) , cv2.COLOR_BGR2HSV)
    img_dim = img_RGB.shape[0]*img_RGB.shape[1]

    # make sure the input image is reshaped and normalized
    input_image = img_HSV.reshape(img_dim, 3)
    input_image = input_image[input_image.sum(axis=(1)) != 0]/255

    # do the same to RGB version for plotting later
    input_image_RGB = img_RGB.reshape(img_dim, 3)
    input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

    # calculate GMM probabilities
    GMMOutput = calculateGMMProbabilities(input_image, clusters)


    print("Conducting image filtering and computations...")
    # image erosion and dilation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    GMMOutput = np.float64(GMMOutput)
    GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 1)
    GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

    # replace image with binary (1,0) for white and black
    GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

    # label image with skimage
    label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)
    
    # display results
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
    for region in regionprops(label_img):
        if region.area >= 1200:
            bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
            if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
                minr, minc, maxr, maxc = region.bbox
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
                ax.add_patch(rect)
                ratio = 493.0884217183345
                barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
                plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
                plt.show()
                print("Test Image Number: " + str(filenameIndex) + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))
    
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

Calculating probabilities for file number: 0...


KeyboardInterrupt: 

In [ ]:
Calculating probabilities for file number: 0...
Conducting image filtering and computations...
Calculating probabilities for file number: 1...
Conducting image filtering and computations...
Test Image Number: 1,  Centroid Row:  557.9893105734174,  Centroid Column:  384.84081869406697,  Distance: 4.49m
Test Image Number: 1,  Centroid Row:  701.8167998018453,  Centroid Column:  448.6254876462939,  Distance: 2.74m
Calculating probabilities for file number: 2...
Conducting image filtering and computations...
Test Image Number: 2,  Centroid Row:  638.7594684541108,  Centroid Column:  480.9151823380519,  Distance: 2.39m
Calculating probabilities for file number: 3...
Conducting image filtering and computations...
Test Image Number: 3,  Centroid Row:  663.9574912891986,  Centroid Column:  396.2994773519164,  Distance: 6.51m
Calculating probabilities for file number: 4...
Conducting image filtering and computations...
Test Image Number: 4,  Centroid Row:  635.2268522979092,  Centroid Column:  441.39581526800913,  Distance: 1.98m
Calculating probabilities for file number: 5...
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
<ipython-input-9-8fe3dda39b54> in <module>
      4     print("Calculating probabilities for file number: " + str(filenameIndex) + "...")
      5     # Reading images as RGB and HSV
----> 6     img_RGB = cv2.cvtColor( cv2.imread(os.path.join(folder,filename), 1) , cv2.COLOR_BGR2RGB)
      7     img_HSV = cv2.cvtColor( cv2.imread(os.path.join(folder,filename), 1) , cv2.COLOR_BGR2HSV)
      8     img_dim = img_RGB.shape[0]*img_RGB.shape[1]

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

In [15]:
img = cv2.cvtColor( cv2.imread("Test_Set/006.png" , 1), cv2.COLOR_BGR2RGB)

In [16]:
img

array([[[163, 154, 136],
        [164, 156, 136],
        [164, 156, 134],
        ...,
        [134, 158, 166],
        [111, 133, 145],
        [ 88, 112, 124]],

       [[162, 154, 136],
        [164, 156, 136],
        [163, 155, 132],
        ...,
        [ 89, 111, 121],
        [ 78,  97, 110],
        [ 89, 109, 123]],

       [[166, 158, 139],
        [164, 156, 136],
        [165, 157, 135],
        ...,
        [ 78, 105, 114],
        [ 99, 121, 133],
        [110, 131, 145]],

       ...,

       [[177,  86,  55],
        [177,  85,  54],
        [179,  86,  53],
        ...,
        [250, 154, 130],
        [254, 156, 133],
        [246, 146, 123]],

       [[172,  81,  50],
        [174,  82,  50],
        [178,  84,  52],
        ...,
        [244, 148, 122],
        [251, 154, 128],
        [244, 146, 119]],

       [[169,  78,  45],
        [172,  80,  46],
        [176,  82,  48],
        ...,
        [240, 145, 117],
        [244, 147, 117],
        [246, 148, 117]]